In [101]:
import pandas as pd
import numpy as np
from garmin_fit_sdk import Decoder, Stream
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import glob
from datetime import datetime
from scipy import signal
from util import lowpass_filter
FIT_EPOCH_S = 631065600

In [102]:
file_list = sorted(glob.glob("./data/*.fit"))
# file_name = "2025-09-14-07-13-58"
file_name = file_list[15]
stream = Stream.from_file(file_name)
decoder = Decoder(stream)
messages, errors = decoder.read(convert_datetimes_to_dates=False)
print(file_name)
errors

./data/2025-09-14-07-58-59.fit


[]

In [103]:
gps_mesgs = messages['gps_metadata_mesgs']
gps_data = pd.DataFrame.from_records(gps_mesgs)
gps_data['utc_timestamp'] = pd.to_datetime((gps_data.utc_timestamp + FIT_EPOCH_S) * 1e9)
gps_data.index = gps_data.timestamp * 1000 + gps_data.timestamp_ms
gps_data.head()

,timestamp,position_lat,position_long,enhanced_altitude,enhanced_speed,utc_timestamp,timestamp_ms,heading,velocity,8,9,10,11,12
12462,12,482483950,-953741811,328.2,0.510,2025-09-14 11:59:12,462,235.67,"[-0.42, -0.28, -0.13]",51,58,77,86,91
12562,12,482483945,-953741795,328.4,0.490,2025-09-14 11:59:12,562,233.80,"[-0.39, -0.28, -0.13]",50,56,76,86,91
12662,12,482483944,-953741772,328.6,0.521,2025-09-14 11:59:12,662,229.91,"[-0.39, -0.33, -0.13]",50,56,75,87,127
12762,12,482483933,-953741771,328.4,0.551,2025-09-14 11:59:12,762,227.38,"[-0.4, -0.37, -0.13]",49,55,74,86,91
12862,12,482483938,-953741749,328.2,0.596,2025-09-14 11:59:12,862,223.78,"[-0.41, -0.43, -0.12]",48,53,72,86,91


In [104]:
calibration_mesgs = messages['three_d_sensor_calibration_mesgs']
calibration_data = { m['sensor_type']: m for m in calibration_mesgs }
accel_cal = calibration_data['accelerometer']
accel_cal

{'timestamp': 2,
 'calibration_factor': 1,
 'calibration_divisor': 2048,
 'level_shift': 32768,
 'offset_cal': [-7, -12, -104],
 'orientation_matrix': [0.0, -1.0, 0.0, 0.0, 0.0, -1.0, -1.0, 0.0, 0.0],
 'sensor_type': 'accelerometer',
 'accel_cal_factor': 1}

In [105]:
accel_raw_list = []
for group in messages['accelerometer_data_mesgs']:
    base_timestamp = group['timestamp'] * 1000 + group['timestamp_ms']
    for i, offset in enumerate(group['sample_time_offset']):
        entry = {
            'timestamp': base_timestamp + offset,
            'x': group['accel_x'][i],
            'y': group['accel_y'][i],
            'z': group['accel_z'][i],
        }
        accel_raw_list.append(entry)
accel_raw = pd.DataFrame.from_records(accel_raw_list)
accel_raw = accel_raw.set_index('timestamp').sort_index()

In [106]:
accel_data = np.array(accel_cal['orientation_matrix']).reshape(3, 3) @ ((accel_raw.to_numpy() \
- accel_cal['level_shift'] - accel_cal['offset_cal']) * \
(accel_cal['calibration_factor'] / accel_cal['calibration_divisor'])).T
accel_data = accel_data.T
accel_data = pd.DataFrame(accel_data, columns=['x', 'y', 'z'], index=accel_raw.index)
fs = 1000 / np.mean(np.diff(accel_data.index))
accel_data['magnitude'] = np.sqrt(accel_filtered.x**2 + accel_filtered.y**2)


accel_filtered = pd.DataFrame(index=accel_data.index, data={
    'x': lowpass_filter(accel_data.x.to_numpy(), 5, fs),
    'y': lowpass_filter(accel_data.y.to_numpy(), 5, fs),
    'z': lowpass_filter(accel_data.z.to_numpy(), 5, fs),
})
accel_filtered['magnitude'] = np.sqrt(accel_filtered.x**2 + accel_filtered.y**2)

In [107]:
px.line(gps_data.enhanced_speed)

In [108]:
px.line(accel_filtered.magnitude)

In [110]:


fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.add_trace(
    go.Scatter(x=gps_data.index, y=gps_data.enhanced_speed, name="v"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=accel_filtered.index, y=accel_filtered.magnitude, name="a"),
    secondary_y=True,
)
fig.show()


In [62]:
f, Pxx = signal.welch(accel_filtered.magnitude.to_numpy(), fs, nperseg=1024)
px.line(pd.DataFrame(index=f, data=dict(PSD=Pxx)))

In [90]:
import json
with open(f'{file_name.replace(".fit", ".json")}', 'w') as f:
    json.dump(messages, f, indent=2, default=str)